## NWB-Datajoint tutorial 1

**Note: make a copy of this notebook and run the copy to avoid git conflicts in the future**

This is the first in a multi-part tutorial on the NWB-Datajoint pipeline used in Loren Frank's lab, UCSF. It demonstrates how to run spike sorting within the pipeline.

If you have not done [tutorial 0](0_intro.ipynb) yet, make sure to do so before proceeding.

Let's start by importing the `nwb_datajoint` package, along with a few others. 

In [60]:
import os
import numpy as np
import datajoint as dj

# dj.config['database.host'] = '127.0.0.1'
# dj.config['database.user'] = 'root'
# dj.config['database.password']= 'simple'

In [61]:


import nwb_datajoint as nd

# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)
os.environ['NWB_DATAJOINT_TEMP_DIR']="/stelmo/nwb/tmp"
os.environ['KACHERY_STORAGE_DIR']="/stelmo/nwb/kachery-storage"


In [62]:
# import tables so that we can call them easily
from nwb_datajoint.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
                                  DataAcquisitionDevice, CameraDevice, Probe,
                                  DIOEvents,
                                  ElectrodeGroup, Electrode, Raw, SampleCount,
                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
                                  SortGroup, SpikeSortingFilterParameters, SpikeSortingArtifactDetectionParameters,
                                  SpikeSortingRecordingSelection, SpikeSortingRecording, 
                                  SpikeSortingWorkspace, 
                                  SpikeSorter, SpikeSorterParameters, SortingID,
                                  SpikeSortingSelection, SpikeSorting, 
                                  SpikeSortingMetricParameters,
                                  ModifySortingParameters, ModifySortingSelection, ModifySorting, 
                                  AutomaticCurationParameters, AutomaticCurationSelection,
                                  AutomaticCuration,
                                  CuratedSpikeSortingSelection, CuratedSpikeSorting,
                                  UnitInclusionParameters,
                                  FirFilter,
                                  IntervalList, SortInterval,
                                  Lab, LabMember, LabTeam, Institution,
                                  BrainRegion,
                                  SensorData,
                                  Session, ExperimenterList,
                                  Subject,
                                  Task, TaskEpoch,
                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery)

In [63]:
nwb_file_name = 'despereaux20191125_.nwb'


Set up the lab members and team information for this sort

In [64]:
#Uncomment to set sort group
#SortGroup().set_group_by_electrode_group(nwb_file_name)

In [65]:
SortGroup.SortGroupElectrode & {'nwb_file_name': nwb_file_name}

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode
despereaux20191125_.nwb,1,1,0
despereaux20191125_.nwb,1,1,1
despereaux20191125_.nwb,1,1,2
despereaux20191125_.nwb,1,1,3
despereaux20191125_.nwb,2,2,4
despereaux20191125_.nwb,2,2,5
despereaux20191125_.nwb,2,2,6
despereaux20191125_.nwb,2,2,7
despereaux20191125_.nwb,3,3,8
despereaux20191125_.nwb,3,3,9


#### Define sort interval
Next, we make a decision about the time interval for our spike sorting. Let's re-examine `IntervalList`.

In [66]:
IntervalList & {'nwb_file_name' : nwb_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,valid_times numpy array with start and end times for each interval
despereaux20191125_.nwb,01_s1,=BLOB=
despereaux20191125_.nwb,02_r1,=BLOB=
despereaux20191125_.nwb,02_r1 lfp band 100Hz,=BLOB=
despereaux20191125_.nwb,03_s2,=BLOB=
despereaux20191125_.nwb,04_r2,=BLOB=
despereaux20191125_.nwb,04_r2 lfp band 100Hz,=BLOB=
despereaux20191125_.nwb,05_s3,=BLOB=
despereaux20191125_.nwb,06_r3,=BLOB=
despereaux20191125_.nwb,06_r3 lfp band 100Hz,=BLOB=
despereaux20191125_.nwb,07_s4,=BLOB=


For our example, let's choose the first 600 seconds of the first run interval (`02_r1`) as our sort interval. To do so, we first fetch `valid_times` of this interval, define our new sort interval, and add this to the `SortInterval` table.

In [67]:
interval_list_name = '02_r1'

In [68]:
interval_list = (IntervalList & {'nwb_file_name' : nwb_file_name,
                            'interval_list_name' : interval_list_name}).fetch1('valid_times')
print(interval_list)

[[1.57470781e+09 1.57471329e+09]]


In [69]:
sort_interval = interval_list[0]
sort_interval_name = interval_list_name
sort_interval = np.copy(interval_list[0]) 
sort_interval[1] = sort_interval[0]+300
sort_interval_name = 'test'

In [70]:
# Check out SortInterval
(SortInterval & {'nwb_file_name' : nwb_file_name})

nwb_file_name name of the NWB file,sort_interval_name name for this interval,sort_interval 1D numpy array with start and end time for a single interval to be used for spike sorting
despereaux20191125_.nwb,02_r1,=BLOB=
despereaux20191125_.nwb,test,=BLOB=


In [71]:
# Specify the required attributes.
# This time, the entries take the form of a dictionary.
#SortInterval.insert1({'nwb_file_name' : nwb_file_name,
#                      'sort_interval_name' : sort_interval_name,
#                      'sort_interval' : sort_interval}, replace=True)

In [72]:
# See results
SortInterval & {'nwb_file_name' : nwb_file_name, 'sort_interval_name': sort_interval_name}

nwb_file_name name of the NWB file,sort_interval_name name for this interval,sort_interval 1D numpy array with start and end time for a single interval to be used for spike sorting
despereaux20191125_.nwb,test,=BLOB=


Now set the filtering parameters. Here we insert the default parameters and a new set of filtering parameters for hippocampal data
|

In [73]:
SpikeSortingFilterParameters().insert_default()
filter_param_dict = SpikeSortingFilterParameters.fetch('filter_parameter_dict')
filter_param_dict = filter_param_dict[0]

In [74]:
filter_param_dict['frequency_min'] = 600
SpikeSortingFilterParameters().insert1({'filter_parameter_set_name': 'franklab_default_hippocampus', 
                                       'filter_parameter_dict' : filter_param_dict}, skip_duplicates=True)

Similarly, we set up the SpikeSortingArtifactParameters which can allow us to remove artifacts from the data
For the moment we just set up a "none" parameter set which will do nothing when used

In [75]:
SpikeSortingArtifactDetectionParameters().insert_default()

Now we set up the recording parameters so we can get the recording extractor

In [76]:
sort_group_id = 2 # use sort group 2
sort_interval_name = 'test'
filter_param_name = 'franklab_default_hippocampus'
artifact_param_name = 'none'
interval_list = '02_r1'
lab_team = 'Loren Frank'

In [77]:
# collect the params
key = dict()
key['nwb_file_name'] = nwb_file_name
key['sort_group_id'] = sort_group_id
key['filter_parameter_set_name'] = filter_param_name
key['sort_interval_name'] = sort_interval_name
key['artifact_parameter_name'] = artifact_param_name
key['interval_list_name'] = interval_list
key['team_name'] = 'Loren Frank'

ssr_key = key

In [113]:
SpikeSortingRecordingSelection()

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,filter_parameter_set_name,artifact_parameter_name name for this set of parameters,interval_list_name descriptive name of this interval list,team_name
despereaux20191125_.nwb,2,test,franklab_default_hippocampus,none,02_r1,Loren Frank


In [79]:
SpikeSortingRecordingSelection.insert1(key, skip_duplicates=True)

In [80]:
SpikeSortingRecording.populate()

Elapsed time for filtered recording extractor setup: 4.286241769790649 sec
Elapsed time for NWB recording extractor create from file: 14.257794618606567 sec
Elapsed time for getting filtered recording extractor: 30.935449838638306 sec
Creating efficient recording: storing link in kachery...
Done creating efficient recording.
Elapsed time for writing filtered h5 recording extractor to /stelmo/nwb/spikesorting/despereaux20191125_.nwb/despereaux20191125_.nwb_test_2_franklab_default_hippocampus_recording.h5v1: 15.313714742660522 sec


Now we need to populate the SpikeSortingWorkspace table to make this recording available via kachery

In [81]:
SpikeSortingRecording()

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,filter_parameter_set_name,sort_interval_list_name descriptive name of this interval list,recording_extractor_path,recording_extractor_object the dictionary that allows kachery to retrieve the extractor
despereaux20191125_.nwb,2,test,franklab_default_hippocampus,despereaux20191125_.nwb_test_2franklab_default_hippocampus_recording,/stelmo/nwb/spikesorting/despereaux20191125_.nwb/despereaux20191125_.nwb_test_2_franklab_default_hippocampus_recording.h5v1,=BLOB=


In [82]:
SpikeSortingWorkspace.populate()


Cannot remove sorting. Sorting not found: S-b7cb9bac4f42
Adding recording: R-2a816b04e6b8
Permissions for lmfrnk@gmail.com set to: {'edit': True}


For our example, we will be using `mountainsort4`.

In [83]:
#SpikeSortingWorkspace().url(key)

In [84]:
SpikeSorter().insert_from_spikeinterface()
SpikeSorterParameters().insert_from_spikeinterface()

In [85]:
sorter_name='mountainsort4'

In [86]:
# Let's look at the default params
ms4_default_params = (SpikeSorterParameters & {'sorter_name' : sorter_name,
                                               'spikesorter_parameter_set_name' : 'default'}).fetch1()
print(ms4_default_params)

{'sorter_name': 'mountainsort4', 'spikesorter_parameter_set_name': 'default', 'parameter_dict': {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'curation': False, 'num_workers': None, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10, 'noise_overlap_threshold': 0.15}}


In [87]:
# Change the default params
param_dict = ms4_default_params['parameter_dict']
# Detect upward downward going spikes
param_dict['detect_sign'] = -1 
#We will sort electrodes together that are within 100 microns of each other
param_dict['adjacency_radius'] = 100
param_dict['curation'] = False
# Turn filter off since we will filter it prior to starting sort
param_dict['filter'] = False
param_dict['freq_min'] = 0
param_dict['freq_max'] = 0
# Turn whiten off since we will whiten it prior to starting sort
param_dict['whiten'] = False
# set num_workers to be the same number as the number of electrodes
param_dict['num_workers'] = 4
param_dict['verbose'] = True
# set clip size as number of samples for 1.33 millisecond
param_dict['clip_size'] = np.int(1.33e-3 * (Raw & {'nwb_file_name' : nwb_file_name}).fetch1('sampling_rate'))
param_dict['noise_overlap_threshold'] = 0



In [88]:
param_dict

{'detect_sign': -1,
 'adjacency_radius': 100,
 'freq_min': 0,
 'freq_max': 0,
 'filter': False,
 'whiten': False,
 'curation': False,
 'num_workers': 4,
 'clip_size': 39,
 'detect_threshold': 3,
 'detect_interval': 10,
 'noise_overlap_threshold': 0,
 'verbose': True}

In [89]:
# Give a unique name here
parameter_set_name = 'franklab_tetrode_hippocampus_30KHz'
SpikeSorterParameters()

sorter_name the name of the spike sorting algorithm,spikesorter_parameter_set_name label for this set of parameters,parameter_dict dictionary of parameter names and values
combinato,default,=BLOB=
hdsort,default,=BLOB=
herdingspikes,default,=BLOB=
ironclust,default,=BLOB=
kilosort,default,=BLOB=
kilosort2,default,=BLOB=
kilosort2_5,default,=BLOB=
kilosort3,default,=BLOB=
klusta,default,=BLOB=
mountainsort4,default,=BLOB=


In [90]:
# Insert
SpikeSorterParameters.insert1({'sorter_name': sorter_name,
                               'spikesorter_parameter_set_name': parameter_set_name,
                               'parameter_dict': param_dict}, skip_duplicates=True)

In [91]:
# Check that insert was successful
#p = (SpikeSorterParameters & {'sorter_name': sorter_name, 'parameter_set_name': parameter_set_name}).fetch1()
p = (SpikeSorterParameters & {'sorter_name': sorter_name}).fetch()
p

array([('mountainsort4', 'default', {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'curation': False, 'num_workers': None, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10, 'noise_overlap_threshold': 0.15}),
       ('mountainsort4', 'franklab_tetrode_hippocampus_30KHz', {'detect_sign': -1, 'adjacency_radius': 100, 'freq_min': 0, 'freq_max': 0, 'filter': False, 'whiten': False, 'curation': False, 'num_workers': 4, 'clip_size': 39, 'detect_threshold': 3, 'detect_interval': 10, 'noise_overlap_threshold': 0, 'verbose': True})],
      dtype=[('sorter_name', 'O'), ('spikesorter_parameter_set_name', 'O'), ('parameter_dict', 'O')])

#### Bringing everything together

We now collect all the decisions we made up to here and put it into `SpikeSortingSelection` table (note: this is different from spike sor*ter* parameters defined above).

In [92]:
key = (SpikeSortingWorkspace & ssr_key).fetch1("KEY")
key['sorter_name'] = sorter_name
key['spikesorter_parameter_set_name'] = 'franklab_tetrode_hippocampus_30KHz'
ss_key = key

In [93]:
# insert
SpikeSortingSelection.insert1(key, skip_duplicates=True)

In [94]:
#(SpikeSortingParameters & {'nwb_file_name' : nwb_file_name, 'sort_interval_name' : sort_interval_name}).delete()

In [95]:
# inspect
(SpikeSortingSelection & {'nwb_file_name' : nwb_file_name})

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,filter_parameter_set_name,sorter_name the name of the spike sorting algorithm,spikesorter_parameter_set_name label for this set of parameters,import_path optional path to previous curated sorting output
despereaux20191125_.nwb,2,test,franklab_default_hippocampus,mountainsort4,franklab_tetrode_hippocampus_30KHz,


#### Running spike sorting
Now we can run spike sorting. As we said it's nothing more than populating another table (`SpikeSorting`) from the entries of `SpikeSortingParameters`.

In [96]:
# Specify entry (otherwise runs everything in SpikeSortingParameters)
# `proj` gives you primary key"
SpikeSorting.populate([(SpikeSortingSelection & {'nwb_file_name' : nwb_file_name, 'sort_interval_name' : sort_interval_name}).proj()])


Whitening recording...
Elapsed time for whitening: 3.2801640033721924 sec

Running spike sorting on {'nwb_file_name': 'despereaux20191125_.nwb', 'sort_group_id': 2, 'sort_interval_name': 'test', 'filter_parameter_set_name': 'franklab_default_hippocampus', 'sorter_name': 'mountainsort4', 'spikesorter_parameter_set_name': 'franklab_tetrode_hippocampus_30KHz'}...
Using 4 workers.
Using tmpdir: /tmp/tmprscjg03u
Num. workers = 4
Preparing /tmp/tmprscjg03u/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=3000000)...
Preparing output...
Done with ms4alg.
Cleaning tmpdir::::: /tmp/tmprscjg03u
mountainsort4 run time 19.51s

Saving sorting results...
Adding sorting: S-ee3365308d96
Writing new NWB file despereaux20191125_41XFEYA1XA.nwb

Done - entry inserted to table.


In [97]:
#SpikeSortingWorkspace().url(key)

#### Define quality metric parameters

We're almost done. There are more parameters related to how to compute the quality metrics for curation. We just use the default options here. 

In [98]:
SpikeSortingMetricParameters()

cluster_metrics_list_name the name for this list of cluster metrics,metric_dict dict of SpikeInterface metrics with True / False elements to indicate whether a given metric should be computed.,metric_parameter_dict dict of parameters for the metrics
franklab_cluster_metrics_09-19-2021,=BLOB=,=BLOB=
franklab_default_cluster_metrics_tmp,=BLOB=,=BLOB=


In [99]:
metric_dict = SpikeSortingMetricParameters().get_metric_dict()
metric_param_dict = SpikeSortingMetricParameters().get_metric_parameter_dict()

In [100]:
for k in metric_dict:
    print(f"'{k}': {metric_dict[k]}\n")

'num_spikes': False

'firing_rate': False

'presence_ratio': False

'isi_violation': False

'amplitude_cutoff': False

'snr': False

'max_drift': False

'cumulative_drift': False

'silhouette_score': False

'isolation_distance': False

'l_ratio': False

'd_prime': False

'noise_overlap': False

'nn_hit_rate': False

'nn_miss_rate': False



In [101]:
metric_dict['noise_overlap'] = True
metric_dict['firing_rate'] = True
metric_dict['num_spikes'] = True
for k in metric_dict:
    print(f"'{k}': {metric_dict[k]}\n")

'num_spikes': True

'firing_rate': True

'presence_ratio': False

'isi_violation': False

'amplitude_cutoff': False

'snr': False

'max_drift': False

'cumulative_drift': False

'silhouette_score': False

'isolation_distance': False

'l_ratio': False

'd_prime': False

'noise_overlap': True

'nn_hit_rate': False

'nn_miss_rate': False



In [102]:
cluster_metrics_list_name = 'franklab_cluster_metrics_09-19-2021'

In [103]:
#(SpikeSortingMetricParameters & {'cluster_metrics_list_name' : cluster_metrics_list_name}).delete()

Add the cluster metrics to the table if they are not there already.

In [104]:
SpikeSortingMetricParameters.insert1({'cluster_metrics_list_name' : cluster_metrics_list_name,
                            'metric_dict' : metric_dict, 
                            'metric_parameter_dict' : metric_param_dict}, skip_duplicates=True)


Add the default Automatic curation parameters

In [105]:
param = AutomaticCurationParameters().get_default_parameters()
AutomaticCurationParameters().insert1({'automatic_curation_parameter_set_name':'none', 
                                      'automatic_curation_parameter_dict': param}, skip_duplicates=True)

Now add an entry to select those parameters for automatic curation of this recording

In [106]:
# first get the sorting ID
acs_key = (SpikeSortingRecording & ssr_key).fetch1('KEY')
acs_key['sorting_id'] = (SpikeSorting & ss_key).fetch1('sorting_id')
acs_key['automatic_curation_parameter_set_name'] = 'none'
acs_key['cluster_metrics_list_name'] = cluster_metrics_list_name
AutomaticCurationSelection.insert1(acs_key, skip_duplicates=True)

Now we populate the Autocuration table, which in this case just computes the metrics and does not add labels.

In [107]:
#AutomaticCuration.delete()

In [108]:
AutomaticCuration.populate(acs_key)

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Number of chunks: 5 - Number of jobs: 24
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Elapsed time for whitening and computing new quality metrics: 17.480809926986694 sec
Writing new NWB file despereaux20191125_HDPIV0IUTC.nwb
Setting unit metrics for sorting S-ee3365308d96


We can now curate the recording using the figurl interface. To do so, we get the figurl link for this recording

In [109]:
SpikeSortingWorkspace().url(ssr_key)

'https://figurl.org/fig?channel=franklab2&figureObject=c7042954b0b12f7e24a0123a5dc003bb711fb8c4&label=despereaux20191125_.nwb_test_2'

Once you're done with manual curation, you can add the units (with an optional new set of metrics) to the final CuratedSortingTable which includes only accepted units.

In [110]:
css_key = (AutomaticCuration & acs_key).fetch1('KEY')
css_key['sorting_id']
css_key['final_cluster_metrics_list_name'] = cluster_metrics_list_name
CuratedSpikeSortingSelection.insert1(css_key, skip_duplicates=True)

In [111]:
CuratedSpikeSorting.populate(css_key)

Found 0 accepted units
{'nwb_file_name': 'despereaux20191125_.nwb', 'sort_group_id': 2, 'sort_interval_name': 'test', 'filter_parameter_set_name': 'franklab_default_hippocampus', 'sorting_id': 'S-ee3365308d96'}: no curation found or no accepted units


In [112]:
CuratedSpikeSorting.Unit()

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,filter_parameter_set_name,sorting_id the sorting id of the sorting that was added,unit_id ID for each unit,label optional label for each unit,noise_overlap noise overlap metric for each unit,nn_isolation isolation score metric for each unit,isi_violation ISI violation score for each unit,firing_rate firing rate,num_spikes total number of spikes
